In [1]:
import pandas as pd
from datasets import load_dataset
hf_data = load_dataset("wykonos/movies")
train_data = hf_data['train']
train_data.to_csv('movies_dataset.csv')
data = pd.read_csv("movies_dataset.csv")
print(data)

Found cached dataset csv (C:/Users/Administrator/.cache/huggingface/datasets/wykonos___csv/wykonos--movies-6c40eec4956b52dc/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/723 [00:00<?, ?ba/s]

            id                                title  \
0       385687                               Fast X   
1       758323                  The Pope's Exorcist   
2       640146    Ant-Man and the Wasp: Quantumania   
3       677179                            Creed III   
4       502356          The Super Mario Bros. Movie   
...        ...                                  ...   
722791  900760  A Incessante Conquista da Escuridão   
722792  957818                  Hollywood Varieties   
722793  648905                 A Man Must Be Strong   
722794  576855                        Stastriptease   
722795  968161                        Gising Sining   

                                           genres original_language  \
0                           Action-Crime-Thriller                en   
1                         Horror-Mystery-Thriller                en   
2                Action-Adventure-Science Fiction                en   
3                                    Drama-Action      

In [2]:
print(data.shape)
data.info() # 20 feature for +700k movies
data.isnull().sum()
data.duplicated().sum()

(722796, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722796 entries, 0 to 722795
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    722796 non-null  int64  
 1   title                 722790 non-null  object 
 2   genres                511934 non-null  object 
 3   original_language     722796 non-null  object 
 4   overview              604198 non-null  object 
 5   popularity            722796 non-null  float64
 6   production_companies  337170 non-null  object 
 7   release_date          670329 non-null  object 
 8   budget                722796 non-null  float64
 9   revenue               722796 non-null  float64
 10  runtime               688346 non-null  float64
 11  status                722796 non-null  object 
 12  tagline               108098 non-null  object 
 13  vote_average          722796 non-null  float64
 14  vote_count            722796 non-null  

1

In [3]:
df = data.drop(["production_companies", "popularity", "budget", "revenue", "status", "recommendations", "runtime", "vote_average", "backdrop_path", "tagline"], axis=1)
df.drop_duplicates(inplace=True)
df.title.duplicated().sum()
df[["title", "release_date"]].duplicated().sum()

2260

In [4]:
df.drop_duplicates(subset=["title","release_date"], inplace=True)
df = df[df.vote_count >= 350].reset_index()
df.isnull().sum()
df.fillna("", inplace = True)
index = df[(df.genres == "") & (df.overview == "")].index
df.drop(index, inplace=True)
df.genres = df.genres.apply(lambda x: " ".join(x.split("-")))
df.keywords = df.keywords.apply(lambda x: " ".join(x.split("-")))
df.credits = df.credits.apply(lambda x: " ".join(x.replace(" ", "").split("-")[:5]))

In [5]:
df["tags"] = df.overview + " "+ df.genres + " "  +df.credits + " " +df.keywords + " " + df.original_language
new_df = df[["id", "title", "tags", "poster_path"]]
new_df.tags = new_df.tags.apply(lambda x:x.lower())

C:\Users\Administrator\AppData\Local\Temp\ipykernel_57792\3127502652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.tags = new_df.tags.apply(lambda x:x.lower())


In [6]:
from nltk.stem.porter import PorterStemmer
new_df = df.copy()

def stem(text):
    ps = PorterStemmer()
    stemmed_words = [ps.stem(word) for word in text.split()]
    return " ".join(stemmed_words)

new_df.loc[:, "tags"] = new_df["tags"].apply(stem)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv = CountVectorizer(stop_words="english", max_features=5000)
vectors = cv.fit_transform(new_df["tags"]).toarray()
similarity = cosine_similarity(vectors)

def recommend(movies):
    movie_index = new_df[new_df.title == movies].index[0]
    distances = similarity[movie_index]
    all_movies_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    for i in all_movies_list:
        print(new_df.iloc[i[0]].title)

C:\Users\Administrator\sheen-sage-notebooks\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [8]:
recommend("12 Angry Men")

Anatomy of a Murder
Find Me Guilty
Marshall
The Verdict
A Time to Kill


In [9]:
import pickle
pickle.dump(new_df, open("movies_model.pkl", "wb"))
pickle.dump(similarity, open("movies_similarity.pkl", "wb"))

In [10]:
movies_list = pickle.load(open("movies_model.pkl", "rb"))
movies_list_title = movies_list["title"].values
similarity = pickle.load(open("movies_similarity.pkl", "rb"))

def recommend(movie):
    movie_index = movies_list[movies_list["title"] == movie].index[0]
    distances = similarity[movie_index]
    sorted_movie_list = sorted(list(enumerate(distances)), reverse=True,
                               key=lambda x: x[1])[1:6]
    recommended_movies = []
    recommended_posters = []
    for i in sorted_movie_list:
        poster_path = movies_list["poster_path"][i[0]]
        recommended_movies.append(movies_list.iloc[i[0]].title)
        recommended_posters.append("https://image.tmdb.org/t/p/original" + poster_path)
    return recommended_movies, recommended_posters

print("Movie Recommender")
selected_movie_name = input("What is the movie name?")
recommendation, movie_posters = recommend(selected_movie_name)

for rec_movie, poster in zip(recommendation, movie_posters):
    print(rec_movie)
    print(poster)

Movie Recommender
Anatomy of a Murder
https://image.tmdb.org/t/p/original/zMxLbSPpToTCc6yK2Gddhx6nHNN.jpg
Find Me Guilty
https://image.tmdb.org/t/p/original/8M0dkX7jOor7exKor1txoUl9Ynu.jpg
Marshall
https://image.tmdb.org/t/p/original/2KfdXsXTCbMie0wB1mSmIX60C2F.jpg
The Verdict
https://image.tmdb.org/t/p/original/iYEuFP1bpj5yOenKliiRFcNO3VE.jpg
A Time to Kill
https://image.tmdb.org/t/p/original/apUSR9WE7lMATBGYhzZ8RnPDYsK.jpg
